In [1]:
# import pandas as pd
# import numpy as np
# import re
#Importing everything from NLP Week 1 - following that as a guide for now
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import random
import pandas as pd
import seaborn as sns
from collections import Counter, defaultdict
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
pd.options.display.max_rows = 100
pd.options.display.width = 150
RANDOM_SEED = 696


In [3]:
WikiLarge_Train_df = pd.read_csv(r'/Users/staceybruestle/Documents/Documents/Education/MADS/Coursework/aSIADS696- Milestone 2/Readability Project/Data/WikiLarge_Train.csv')#, \

In [5]:
WikiLarge_Train_df['parsed_list'] = 0
WikiLarge_Train_df['All_Characters'] = 0
WikiLarge_Train_df['Is_Int'] = 0
for i, row in WikiLarge_Train_df.iterrows():
        temp = row['original_text']
        WikiLarge_Train_df['parsed_list'].at[i] = re.split(r'(\s)', temp)
        if temp.isalpha():
                WikiLarge_Train_df['All_Characters'].at[i] = 1
        try:
                int(temp)
                WikiLarge_Train_df['Is_Int'].at[i] = 1
        except:
                pass
#parsed_list = is the sentence split on whitespace, retaining each whitespace as an element in the list
print(WikiLarge_Train_df.head(1))

In [9]:
train_df, dev_df, test_df = np.split(WikiLarge_Train_df.sample(frac=1, random_state= RANDOM_SEED), 
                       [int(.8*len(WikiLarge_Train_df)), int(.9*len(WikiLarge_Train_df))], axis = 0)
#Make list of labels
y_train = list(train_df.label)
y_dev = list(dev_df.label)
y_test = list(test_df.label)
# print(train_df.shape)
# print(dev_df.shape)
# print(test_df.shape)
# 333414, 41677, 41677 all with 5 columns

In [11]:
ws_tokens = Counter()  #split on whitespace
alpha_ws_tokens = Counter() #split on white space, keeps each word once like a set, only keeping tokens containing alphanumeric chars
alpha_re_tokens = Counter() # split on white space keeping every portion of each token without other chars
                            # example: "don't" is included in ws_tokens, not in alpha_ws_tokens, and "don", "t" is in alpha_re_tokens
count = 0

for sent in tqdm(train_df.original_text):
    alphanumeric_list = []
    alpha_re_list = []
    sentlist = sent.split()

    ws_tokens.update(sentlist)

    for element in sentlist:
        #alphanumeric_regex = re.compile(r'[a-zA-Z0-9]+')
        alphanumeric_regex = re.compile(r'\w+')
        if re.fullmatch(alphanumeric_regex, element) != None:
            alphanumeric_list.append(element)
            #print("element:", element, "\nalpha_ws_list:",alpha_ws_list)

    alpha_re_list = re.findall(alphanumeric_regex, sent)
    alpha_ws_tokens.update(alphanumeric_list)
    alpha_re_tokens.update(alpha_re_list)
    # ws_tokens results in 180497 tokens
    # alpha_ws_tokens results in 150320 tokens
    # alpha_re_tokens results in 155812

100%|██████████| 333414/333414 [00:08<00:00, 39238.84it/s]


In [1]:
# top_50 = alpha_re_tokens.most_common(50)
# #hidden tests are within this cell
# top_50

Convert text to features

In [15]:
vectorizer = TfidfVectorizer(min_df=500, stop_words= 'english')
X_train = vectorizer.fit_transform(train_df.original_text)
print(X_train.shape)

(333414, 1099)


Fit the classifier

In [17]:
clf = LogisticRegression(random_state=RANDOM_SEED, max_iter=500, multi_class='ovr', solver= 'newton-cholesky')
#clf.fit(X_train[0:10000,:], y_train[0:10000])
clf.fit(X_train, y_train)

# Generate the dev data
X_dev = vectorizer.transform(dev_df.original_text)

LogisticRegression(max_iter=500, multi_class='ovr', random_state=696,
                   solver='newton-cholesky')

Create the Dummy Classifiers

In [19]:
uniform_dummy = DummyClassifier(strategy='uniform', random_state = RANDOM_SEED, constant=None)
most_freq_dummy = DummyClassifier(strategy='most_frequent', random_state = RANDOM_SEED, constant=None)

uniform_dummy.fit(X_train, y_train)
most_freq_dummy.fit(X_train, y_train)

# Generate the predictions
lr_tiny_dev_preds = clf.predict(X_dev)
rand_dev_preds = uniform_dummy.predict(X_dev)
mf_dev_preds = most_freq_dummy.predict(X_dev)

# Score the predictions
lr_f1 = f1_score(y_dev, lr_tiny_dev_preds, average='macro')
rand_f1 = f1_score(y_dev, rand_dev_preds, average='macro')
mf_f1 = f1_score(y_dev, mf_dev_preds, average='macro')
print(lr_f1)
print(rand_f1)
print(mf_f1)


DummyClassifier(random_state=696, strategy='most_frequent')

Create a function for the steps so we can run it for various amounts of data to see the difference

In [22]:
def train_and_score(train_df, dev_df):
    # Fit a new TfidfVectorizer
    #vectorizer = TfidfVectorizer(min_df=25, stop_words= 'english')
    vectorizer = TfidfVectorizer(min_df=25, max_df= .9)
    X_train = vectorizer.fit_transform(train_df.original_text)

    #Get the labels
    y_train = list(train_df.label)

    #Fit the data to a Logistic Regression Classifier (in this example a subset of 10k)
    #clf = LogisticRegression(random_state=RANDOM_SEED, multi_class='auto', solver= 'lbfgs')
    clf = LogisticRegression(random_state=RANDOM_SEED, max_iter=500, C=.1, multi_class='ovr', solver= 'newton-cholesky')
    clf.fit(X_train, y_train)

    # Generate the dev data
    X_dev = vectorizer.transform(dev_df.original_text)
    y_dev = list(dev_df.label)

    # Generate the predictions
    lr_tiny_dev_preds = clf.predict(X_dev)

    # Score the predictions
    f1 = f1_score(y_dev, lr_tiny_dev_preds, average='macro')

    # the function returns the macro-averaged F1 score on the dev data
    return f1

def change_in_performance(training_sizes, train_df, dev_df):
    random.seed(RANDOM_SEED)
    f1_scores = []
    for training_size in tqdm(training_sizes):
        f1_scores.append(train_and_score(train_df.head(training_size), dev_df))
    return(f1_scores)

training_sizes = [1000, 10000, 50000, 100000, 333414]
performace_f1_scores = change_in_performance(training_sizes, train_df, dev_df)

100%|██████████| 5/5 [01:32<00:00, 18.46s/it]


Try a uni, bi, and trigram vectorizer

In [27]:
trigram_vectorizer = TfidfVectorizer(stop_words='english', min_df=25, ngram_range=(1,3))
X_train = trigram_vectorizer.fit_transform(train_df.original_text)
print(X_train.shape)
clf = LogisticRegression(random_state=RANDOM_SEED, multi_class='ovr', solver= 'newton-cholesky')
clf.fit(X_train, y_train)
#featurize the dev data
X_dev = trigram_vectorizer.transform(dev_df.original_text)
#predict the dev data and score
lr_trigram_dev_preds = clf.predict(X_dev)
lr_f1 = f1_score(y_dev, lr_trigram_dev_preds, average='macro')
print(lr_f1)

(333414, 23797)


min_df @ 500, ngram_range(1,3), multi_class='ovr', solver = 'newton-cholesky' (didn't take too long) scored 0.6451618658
min_df @  25, ngram_range(1,3), multi_class='ovr', solver = 'newton-cholesky' (6m 19.1s) scored 0.6880169450552442